In [25]:
from selenium import webdriver
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools
import csv
import json
from collections import Counter
import jwt
import datetime
import requests
from lxml import html
import re
import dask.dataframe as dd # pip install dask

NaN = np.nan


headers = {
    'User-Agent': 'Mie Damgaard',
    'mail' : 'rks516@alumni.ku.dk'
}

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [26]:
POST_df = pd.read_csv('CSV_input/postnumre.csv', encoding='ISO-8859-1', delimiter=';', header=None)
STRAF22_df = pd.read_csv('CSV_input/STRAF22.csv', encoding='ISO-8859-1', delimiter=';', header=None)
INDKF122_df = pd.read_csv('CSV_input/kommuneindkomst.csv', encoding='ISO-8859-1', delimiter=';', header=None)
AREALDK_df = pd.read_csv('CSV_input/AREALDK.csv', encoding='ISO-8859-1', delimiter=';', header=None)
FOLK1E_df = pd.read_csv('CSV_input/FOLK1E.csv', encoding='ISO-8859-1', delimiter=';', header=None)
AFSTB4_df = pd.read_csv('CSV_input/AFSTB4.csv', encoding='ISO-8859-1', delimiter=';', header=None)

In [27]:
#SKIFTE TIL FÆRRE DECIMALER
my_dataframes = [POST_df, STRAF22_df, INDKF122_df, AREALDK_df, FOLK1E_df, AFSTB4_df]

def change_to_32(dataframe):
    dataframe = dataframe.astype({col: 'float32' for col in dataframe.select_dtypes(include='float64').columns})
    dataframe = dataframe.astype({col: 'int32' for col in dataframe.select_dtypes(include='int64').columns})
    return dataframe

my_dataframes = [change_to_32(df) for df in my_dataframes]
postnummer_df, STRAF22_df, INDKF122_df, AREALDK_df, FOLK1E_df, AFSTB4_df = my_dataframes

In [28]:
#POSTNUMMER
POST_df = pd.read_csv('CSV_input/postnumre.csv', encoding='ISO-8859-1', delimiter=';', header=None)
POST_df = POST_df.iloc[:, 2] # Gem kolonne med string: 'postnummer, bynavn, kommune'
POST_df = POST_df.str.extract(r'(\d+)\s+(.+)\s\((.+)\)')  # Split kolonnen til 'Postal code', 'By_fra_DST', 'Kommune_fra_DST'
POST_df.columns = ['Postal code', 'By_fra_DST', 'Kommune_fra_DST']
POST_df = POST_df.dropna(how='all')
POST_df['Postal code'] = POST_df['Postal code'].astype('int32')

In [29]:
#STRAF22_df
STRAF22_df = pd.read_csv('CSV_input/STRAF22.csv', encoding='ISO-8859-1', delimiter=';', header=None)
STRAF22_df.columns = ['lovovertrædelse', 'år', 'By_fra_DST', 'antal anmeldelser', 'antal sigtelser' ]
STRAF22_df = STRAF22_df.drop(columns=['lovovertrædelse', 'år'])
STRAF22_df.drop(STRAF22_df[STRAF22_df['By_fra_DST'].isin(['Region Hovedstaden', 'Region Sjælland', 'Region Midtjylland', 'Uoplyst kommune', 'Region Syddanmark', 'Region Nordjylland'])].index, inplace=True)

In [30]:
#INDKF122_df
INDKF122_df = pd.read_csv('CSV_input/kommuneindkomst.csv', encoding='ISO-8859-1', delimiter=';', header=None)
INDKF122_df = INDKF122_df.iloc[:, -2:]
INDKF122_df.columns = ['By_fra_DST', 'Familiernes gennemsnitlige indkomst indkomst'] 

In [31]:
#AREALDK
AREALDK_df = pd.read_csv('CSV_input/AREALDK.csv', encoding='ISO-8859-1', delimiter=';', header=None)
AREALDK_df = AREALDK_df.iloc[:, -2:]
AREALDK_df.columns = ['By_fra_DST', 'm2 parker, sportsanlæg og rekreative omr pr indbygger']
AREALDK_df

,By_fra_DST,"m2 parker, sportsanlæg og rekreative omr pr indbygger"
0,Region Hovedstaden,40.0
1,København,16.0
2,Frederiksberg,9.8
3,Dragør,69.8
4,Tårnby,29.0
...,...,...
99,Morsø,105.1
100,Rebild,84.0
101,Thisted,85.1
102,Vesthimmerlands,169.6


In [32]:
#FOLK1E_df
FOLK1E_df = pd.read_csv('CSV_input/FOLK1E.csv', encoding='ISO-8859-1', delimiter=';', header=None)
FOLK1E_df.columns = ['alder', 'enhed', 'tidspunkt', 'By_fra_DST', 'Indbyggertal i by', 'personer med dansk oprindelse', 'Indvandrere fra vestlige lande', 'Indvandrere fra ikke vestlige lande', 'Efterkommere fra vestlige lande', 'Efterkommere fra ikke vestlige lande']
FOLK1E_df['andel_dansk_opr']=FOLK1E_df['personer med dansk oprindelse']/FOLK1E_df['Indbyggertal i by']
FOLK1E_df['andel_vestlig_opr']=(FOLK1E_df['Indvandrere fra vestlige lande'] + FOLK1E_df['Efterkommere fra vestlige lande'])/FOLK1E_df['Indbyggertal i by']
FOLK1E_df['andel_ikke_vestlig_opr']=(FOLK1E_df['Indvandrere fra ikke vestlige lande'] + FOLK1E_df['Efterkommere fra ikke vestlige lande'])/FOLK1E_df['Indbyggertal i by']
FOLK1E_df = FOLK1E_df.drop(columns = ['Indbyggertal i by', 'alder', 'enhed', 'tidspunkt', 'personer med dansk oprindelse', 'Indvandrere fra vestlige lande', 'Indvandrere fra ikke vestlige lande', 'Efterkommere fra vestlige lande', 'Efterkommere fra ikke vestlige lande'])
FOLK1E_df

,By_fra_DST,andel_dansk_opr,andel_vestlig_opr,andel_ikke_vestlig_opr
0,Region Hovedstaden,0.769098,0.079749,0.151153
1,København,0.717574,0.116602,0.165823
2,Frederiksberg,0.796589,0.091334,0.112077
3,Dragør,0.911877,0.043093,0.045030
4,Tårnby,0.840526,0.047000,0.112474
...,...,...,...,...
99,Morsø,0.928153,0.028300,0.043547
100,Rebild,0.934662,0.025224,0.040114
101,Thisted,0.903193,0.045747,0.051060
102,Vesthimmerlands,0.893487,0.048991,0.057523


In [42]:
#AFSTB4_df
AFSTB4_df = pd.read_csv('CSV_input/AFSTB4.csv', encoding='ISO-8859-1', delimiter=';', header=None)
AFSTB4_df.columns = ['I alt', 'kategori', 'By_fra_DST', 'Gennemsnitlig pendligafstand']
AFSTB4_df = AFSTB4_df.drop(columns =['I alt', 'kategori'])
AFSTB4_df

,By_fra_DST,Gennemsnitlig pendligafstand
0,Region Hovedstaden,15.5
1,Landsdel Byen København,12.7
2,København,12.6
3,Frederiksberg,12.4
4,Dragør,18.1
...,...,...
110,Morsø,19.6
111,Rebild,30.2
112,Thisted,22.5
113,Vesthimmerlands,25.7


In [44]:
# Start med at merge to DataFrames og fortsæt derfra
merged_df = pd.merge(INDKF122_df, STRAF22_df, on='By_fra_DST', how='right')
merged_df = merged_df.merge(INDKF122_df, on='By_fra_DST', how='right')
merged_df = merged_df.merge(AREALDK_df, on='By_fra_DST', how='right')
merged_df = merged_df.merge(FOLK1E_df, on='By_fra_DST', how='right')
merged_df = merged_df.merge(AFSTB4_df, on='By_fra_DST', how='right')

# Resultat
print(len(merged_df))
merged_df.drop(merged_df[merged_df['By_fra_DST'].isin(['Landsdel Østjylland','Landsdel Fyn','Christiansø', 'Landsdel Nordsjælland','Landsdel Nordjylland', 'Landsdel Vestjylland','Landsdel Østjylland	','Landsdel Byen København','Landsdel Vest- og Sydsjælland','Landsdel Sydjylland' ,'Landsdel Østsjælland', 'Landsdel Bornholm', 'Landsdel Københavns omegn','Region Hovedstaden', 'Region Sjælland', 'Region Midtjylland', 'Uoplyst kommune', 'Region Syddanmark', 'Region Nordjylland'])].index, inplace=True)
merged_df = merged_df.reset_index()
print(len(merged_df))

115
98


In [46]:
merged_df

,index,By_fra_DST,Familiernes gennemsnitlige indkomst indkomst_x,antal anmeldelser,antal sigtelser,Familiernes gennemsnitlige indkomst indkomst_y,"m2 parker, sportsanlæg og rekreative omr pr indbygger",andel_dansk_opr,andel_vestlig_opr,andel_ikke_vestlig_opr,Gennemsnitlig pendligafstand
0,2,København,537084.0,67164.0,15055.0,537084.0,16.0,0.717574,0.116602,0.165823,12.6
1,3,Frederiksberg,651156.0,6772.0,1083.0,651156.0,9.8,0.796589,0.091334,0.112077,12.4
2,4,Dragør,823921.0,416.0,57.0,823921.0,69.8,0.911877,0.043093,0.045030,18.1
3,5,Tårnby,608764.0,3268.0,932.0,608764.0,29.0,0.840526,0.047000,0.112474,13.6
4,7,Albertslund,515452.0,1735.0,390.0,515452.0,44.4,0.668760,0.046179,0.285061,13.6
...,...,...,...,...,...,...,...,...,...,...,...
93,110,Morsø,536268.0,459.0,169.0,536268.0,105.1,0.928153,0.028300,0.043547,19.6
94,111,Rebild,674931.0,591.0,121.0,674931.0,84.0,0.934662,0.025224,0.040114,30.2
95,112,Thisted,555421.0,1085.0,421.0,555421.0,85.1,0.903193,0.045747,0.051060,22.5
96,113,Vesthimmerlands,555936.0,1967.0,1146.0,555936.0,169.6,0.893487,0.048991,0.057523,25.7


In [48]:
merged_df.to_csv('CSV_output/DST.csv', index=False)